In [169]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [204]:
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000 # the number of training iterations
eval_interval = 200 # how often to evaluate the loss
learning_rate = 1e-3 # learning rate
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200 # number of iterations for evaluation, from random samples
embed_size = 32 # embedding size
split_ratio = 0.9 # training, val split ratio
# ------------

In [171]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-14 22:41:51--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2026-01-14 22:41:51 (9.32 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [172]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [173]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [174]:
class modeldata:
    def __init__(self, data, tag):
        self.data =data
        self.tag = tag

In [175]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(split_ratio*len(data)) # first 90% will be train, rest val
train_data = modeldata(data[:n], "train")
val_data = modeldata(data[n:], "val")

In [210]:
class dataloaderlite:
    def __init__(self, data, block_size, batch_size, shuffle=True, tag=None):
        self.data = data
        self.L = block_size
        self.B= batch_size
        self.current_pos = 0
        self.shuffle = shuffle
        self.N = len(data)
        self.tag = tag

    def __iter__(self):
        return self

    def __next__(self):
        if self.shuffle:
            # shuffle mode: sample random batches
            idx = torch.randint(0,self.N - self.L - 1, (self.B,))
        else:
            # sequential mode: get batches in order, mainly for eval
            if self.current_pos + self.B * self.L >= self.N:
                self.current_pos = 0  # reset pointer if we reach the end
                raise StopIteration
            idx = torch.arange(self.current_pos, self.current_pos + self.B * self.L, self.L)
            self.current_pos += self.B * self.L
        # x is from i -> i + L token index
        # y is from 1+1 -> i+ 1 + L token index    
        x = torch.stack([self.data[i:i+self.L] for i in idx])
        y = torch.stack([self.data[i+1:i+self.L+1] for i in idx])    
        return x.to(device), y.to(device)

In [217]:
class BaselineModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super().__init__()
        self.embed_table = nn.Embedding(vocab_size, embed_size)
        self.FFN = nn.Linear(embed_size, vocab_size)
    
    def forward(self, x_id_matrix, y_id_matrix=None):
        # id_matrix: (B, L) is the input token indices for B batches, each batch has sequence length L
        # d: the embed size
        # v: the vocab size
        B, L = x_id_matrix.shape
        token_emb = self.embed_table(x_id_matrix) # (B, L, d)
        logits = self.FFN(token_emb) # (B, L, vocab_size)
        if y_id_matrix is None:
            loss = None
        else:
            B, L, v = logits.shape
            # reshape the logits and targets to compute the cross-entropy loss
            logits = logits.view(B*L, v)
            targets = y_id_matrix.view(B*L)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, x_id_matrix, max_new_tokens):
        B, L = x_id_matrix.shape
        for _ in range(max_new_tokens):
            logits, _ = self(x_id_matrix)
            # focus only on the last time step
            logits = logits[:, -1, :] # (B, vocab_size)
            probs = F.softmax(logits, dim=-1) # (B, vocab_size)
            # sample from the distribution
            next_id = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            x_id_matrix= torch.cat((x_id_matrix, next_id), dim=1) # (B, L+1)
        return x_id_matrix
       

In [211]:
train_loader = dataloaderlite(train_data.data, block_size, batch_size, shuffle=True, tag="train")
val_loader = dataloaderlite(val_data.data, block_size, batch_size, shuffle=False, tag="val")

In [218]:
model = BaselineModel(vocab_size, embed_size)
model = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [219]:
@torch.no_grad()
def loss_estimation(model):
    output = {}
    model.eval()
    train_data_loader_estimation = dataloaderlite(train_data.data, block_size, batch_size, shuffle=True, tag="train")
    val_data_loader_estimation = dataloaderlite(val_data.data, block_size, batch_size, shuffle=True, tag="val")
    for data_obj in [train_data_loader_estimation, val_data_loader_estimation]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = data_obj.__next__()
            logits, loss = model(x, y)
            losses[k] = loss.item()
        output[data_obj.tag] = losses.mean().item()
    model.train()
    return output

In [220]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = loss_estimation(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = train_loader.__next__()

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3942, val loss 4.3823
step 200: train loss 3.1696, val loss 3.1757
step 400: train loss 2.7682, val loss 2.7706
step 600: train loss 2.6530, val loss 2.6592
step 800: train loss 2.5928, val loss 2.6089
step 1000: train loss 2.5612, val loss 2.5734
step 1200: train loss 2.5513, val loss 2.5695
step 1400: train loss 2.5368, val loss 2.5392
step 1600: train loss 2.5143, val loss 2.5495
step 1800: train loss 2.5139, val loss 2.5302
step 2000: train loss 2.5098, val loss 2.5331
step 2200: train loss 2.4939, val loss 2.5132
step 2400: train loss 2.5058, val loss 2.5123
step 2600: train loss 2.4910, val loss 2.5127
step 2800: train loss 2.4988, val loss 2.4984
step 3000: train loss 2.4951, val loss 2.5170
step 3200: train loss 2.4908, val loss 2.5022
step 3400: train loss 2.4805, val loss 2.5053
step 3600: train loss 2.4831, val loss 2.4975
step 3800: train loss 2.4843, val loss 2.4939
step 4000: train loss 2.4745, val loss 2.5024
step 4200: train loss 2.4786, val loss 2.

In [221]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_context_ids = model.generate(context, max_new_tokens=1000)
print(decode(generated_context_ids[0].tolist()))



Pe derer n, y wir llurest me.
RARINE:

Pr thy hakr.
Ais
I was sor w we, INore t w myomad be au tut h and owooun Thayoromeaigeremell brolloue thard t h biwhthyodethiove sathenorof urnt, we t helist,
thilerisone dwhat, cororeand omielde bertore, winowas ot po y, thor iou thepl, lllllenout ougngeasene ke men'sth, stororothe
Witrof orwhoupendo ARhe y cen LO:
ARY
O:
NI Hove.
Moup atend mane hes.

CAre and h MPr afas nllaitese hatwhate veer INEThan, min, ar me l; w ha m, lanbe nkem.
Todsceartherr mp I
Con ind
Hou tr h y t fonds wor ht hiu dpithithereepld pe st. krdyo aro hr y ircelom.
Th, dee hesthindy, my,



AD,
 r, g;
Thed; sth, tothen lon muge heame ll, tlid beyout alldis ndor
JUSe.
Ag ke, t mis, n co s find en: ne aso ad y y wee hitoowin sbye southy, buce g, bucowhachenofe-
NT:
Fovot ndganoblot l
Memyolo?
KIXIUCar batheasl, r hefinout wane t s t lag matomel ile h mirodest:
MIO f fowintoof we?
Heixt, fe thieathat bed he wors tr
INGLmplirer, ame s aprtor he
Hersea wicom thorajer, icole i

In [ ]:
class GPT(nn.Module):
    def __init__(self, vocab_size, embed_size, L, n_layers, num_heads):
        super().__init__()
        self.token_emb_table = nn.Embedding(vocab_size, embed_size)
        self.pos_emb_table = nn.Embedding(L, embed_size)
        self.attention_blocks = nn.ModuleList(MultiHeadAttention(num_heads)
                                               for _ in range(n_layers))
        self.FFN = nn.Linear(embed_size, 4*embed_size)
        self.LM = nn.Linear(4*embed_size, vocab_size)
        self.L = L
        self.n_layers = n_layers
    
    def forward(self, x_id_matrix, y_id_matrix=None):
        B, L = x_id_matrix.shape
        token_emb = self.token_emb_table(x_id_matrix)
        pos_emb = self.pos_emb_table(torch.arange(self.L))
        x = token_emb + pos_emb
        for block in self.attention_blocks:
          x = x + block(x)
        x = x + self.FFN(x)
        logits = x + self.LM(x)
        if y_id_matrix == None:
            loss = 0.0       
        logits = logits.view(B*L, self.vocab)
        targets = y_id_matrix.view(B*L)
        loss = F.cross_entropy(logits, targets)
        return logits, loss
    

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.attention_heads = nn.ModuleList(MaskedAttention(num_heads) 
                                             for _ in range(num_heads)) 
        
    
    def forward(self, x):
        chuncks = []
        for attention_head in self.attention_heads: 
            out = attention_head(x) # (B, L, d/H)
            chuncks.append(out)
        final_out = torch.cat(chuncks,dim=-1)
        return final_out # (B, L, d)
        

In [ ]:
class MaskedAttention(nn.Module):
    def __init__(self, num_heads):
        super().__init__()
        self.q_proj = nn.Linear(embed_size, embed_size/num_heads)
        self.k_proj = nn.Linear(embed_size, embed_size/num_heads)
        self.v_proj = nn.Linear(embed_size, embed_size/num_heads)
        self.FFN = nn.Linear()
        self.ln1 = nn.LayerNorm()
        self.ln2 = nn.LayerNorm()

    def forward(self, x):
    
     
        return attention_output 

In [231]:
chuncks = []
a = torch.randint(0, 10, (2,3))
for _ in range(4):
    chuncks.append(a)
torch.cat(chuncks, dim =-1)


tensor([[6, 5, 5, 6, 5, 5, 6, 5, 5, 6, 5, 5],
        [9, 7, 1, 9, 7, 1, 9, 7, 1, 9, 7, 1]])